In [1]:
import numpy as np
import scipy.linalg
import cvxpy as cp
from numpy.linalg import matrix_power

A Robust Learning Approach for Regression Models Based on Distributionally Robust Optimization

In [2]:
def dlqr(A,B,Q,R):
    """
    Calculates the optimal gain matrix K for system
    x[k+1] = A x[k] + B u[k]
    with cost = sum x[k].T*Q*x[k] + u[k].T*R*u[k], such that
    u[k] = -K x[k]
    
    Returns: 
        -K
    """
    # first, solve the ricatti equation
    P = np.matrix(scipy.linalg.solve_discrete_are(A, B, Q, R))
    # compute the LQR gain
    K = np.matrix(scipy.linalg.inv(B.T*P*B+R)*(B.T*P*A))
    return -K

In [3]:
def disc_linear_system(A,B,delta_t):
    '''
    Discrete a linear system with implicit Euler
    x[k+1] = (I - delta_t * A)^{-1} @ x[k] + (I - delta_t * A)^{-1} @ (delta_t * B) @ u[k]
    
    Returns:
        Ak
        Bk
    
    ''' 
    Nx = np.shape(A)[0]
    Ix = np.identity(Nx)
    
    Ak = np.linalg.inv(Ix - delta_t * A)
    Bk = np.linalg.inv(Ix - delta_t * A) @ (delta_t * B)
    
    
    return Ak, Bk

In [4]:
def RK4_np(f, x, u, t, h):
    """
    Runge-Kutta 4th order solver using numpy array data type.

    Args:
        f: A function returning first order ODE in 2D numpy array (Nx x 1).
        x: Current value (list or numpy array). 
        t: Current time.
        h: Step length.
    Returns:
        x_next: Vector of next value in 2D numpy array (Nx x 1)
    """
    x = np.reshape(x, (np.shape(x)[0], -1))    # Reshape x to col vector in np 2D array
    k1 = f(t, x, u)
    k2 = f(t + h / 2, x + h / 2 * k1, u)
    k3 = f(t + h / 2, x + h / 2 * k2, u)
    k4 = f(t + h, x + h * k3, u)
    x_next = x + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    return x_next

In [5]:
def mass_string_ode(t, x, u):
    m = 2 #[kg]
    k1 = 3 # [N/m]
    k2 = 2 # [N/m]
    
    A = np.array([[0,1],[-k2/m, -k1/m]])
    B = np.array([[0],[1/m]])
    
    dot_x = A @ x + B @ u
    
    return dot_x

In [6]:
def stack_system(A, B, C, D, E, x_init, N):
    '''
    Stack system matrix for N prediction horizon
    
    x_next = A x_k + B u_k + C w_k
    y_k = D x_k + E w_k
    
    '''

    n = np.shape(A)[1]    #  Dimension of state
    m = np.shape(B)[1]    #  Dimension of input
    d = np.shape(C)[1]    #  Dimension of disturbance
    r = np.shape(D)[0]    #  Dimension of output
    
#     print(Nx,Nu,Nw,Ny)
    Nx = (N+1) * n
    Nu = N * m
    Ny = N * r
    Nw = N * d
    
    Ax = np.zeros([Nx,n])
    Bx = np.zeros([Nx,Nu])
    Cx = np.zeros([Nx,Nw])
    Ay = np.zeros([Ny,n])
    By = np.zeros([Ny,Nu])
    Cy = np.zeros([Ny,Nw])
    Ey = np.zeros([Ny,Nw])
    Cx_tilde = np.zeros([Nx, Nw + 1])
    Cy_tilde = np.zeros([Ny + 1, Nw + 1])
    Ey_tilde = np.zeros([Ny + 1, Nw + 1])
    D_tilde = np.zeros([Ny + 1, Nx])
#     H

# Ax
    for i in range(N+1):
        Ax[i*n:(i+1)*n,:] = matrix_power(A,i)
# Bx
    for i in range(N):
        mat_temp = B
        for j in range (i+1):
            Bx[(i + 1) * n : (i + 2) * n, (i-j) * m: (i-j+1) * m ] = mat_temp
            mat_temp = A @ mat_temp
# Cx
    for i in range(N):
        mat_temp = C
        for j in range (i+1):
            Cx[(i + 1) * n : (i + 2) * n, (i-j) * d: (i-j+1) * d ] = mat_temp
            mat_temp = A @ mat_temp
# Ay
    for i in range(N):
        Ay[i*r:(i+1)*r,:] = D @ matrix_power(A,i) 
# By
    for i in range(N):
        mat_temp = B
        for j in range (i+1):
            By[(i + 1) * r : (i + 2) * r, (i-j) * m: (i-j+1) * m ] = D @ mat_temp
            mat_temp = A @ mat_temp
# Cy
    for i in range(N):
        mat_temp = C
        for j in range (i+1):
            Cy[(i + 1) * r : (i + 2) * r, (i-j) * d: (i-j+1) * d ] = D @ mat_temp
            mat_temp = A @ mat_temp
# Ey
    for i in range(N):
        Ey[i * r : (i+1)*r, i * d : (i+1) * d] = E
# Cx_tilde
    Cx_tilde[ : , 0 : 1 ] = Ax @ x_init
    Cx_tilde[ : , 1 : ] = Cx
# Cy_tilde
    Cy_tilde[ r : , 0 : 1 ] = Ay @ x_init
    Cy_tilde[ r : , 1 : ] = Cy
# Ey_tilde
    Ey_tilde[0, 0] = 1
    Ey_tilde[1:, 1:] = Ey
# D_tilde
    for i in range(N):
        D_tilde[1 + i * r : 1 + (i+1) * r, i * d : (i+1) * d] = D
        
    return n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde 

Continuous-time system

$\begin{aligned}\left[\begin{array}{c}\dot{x}_{1}(t) \\ \dot{x}_{2}(t)\end{array}\right] &=\left[\begin{array}{cc}0 & 1 \\ -k_{2} / m & -k_{1} / m\end{array}\right]\left[\begin{array}{l}x_{1}(t) \\ x_{2}(t)\end{array}\right]+\left[\begin{array}{c}0 \\ 1 / m\end{array}\right] u(t) \\ y(t) &=\left[\begin{array}{ll}1 & 0\end{array}\right]\left[\begin{array}{l}x_{1}(t) \\ x_{2}(t)\end{array}\right] \end{aligned}$

In [7]:
m = 2 #[kg]
k1 = 3 # [N/m]
k2 = 2 # [N/m]

para = [m, k1, k2]

x_init = np.array([[-2],[0]])

In [8]:
A = np.array([[0,1],[-k2/m, -k1/m]])
B = np.array([[0],[1/m]])
delta_t = 0.1

In [9]:
Ak,Bk = disc_linear_system(A,B,delta_t)

In [10]:
Ak

array([[ 0.99137931,  0.0862069 ],
       [-0.0862069 ,  0.86206897]])

In [11]:
Bk

array([[0.00431034],
       [0.04310345]])

In [12]:
Ck = np.array([[1e-3, 0],[0, 0]])
D = np.array([[1, 0]])
E = np.array([[0,1e-3]])

N = 5
n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde  = stack_system(Ak,Bk,Ck,D,E, x_init, N)

In [13]:
Q = np.diag([10,1])
Qf = np.diag([15,1])
R = np.diag([1])

In [14]:
neg_K = dlqr(Ak,Bk,Q,R)

In [15]:
neg_K

matrix([[-1.64465128, -1.02359576]])

In [16]:
N_sim = 100


t0 = 0

xk = x_init
uk = 0
t = t0
h = delta_t

x_list = []
x_list += [xk.flatten().tolist()]
u_list = []
for i in range(N_sim):
    wk = np.random.randn(d,1)    
    uk = neg_K @ xk
    u_list +=  uk.flatten().tolist() 
    x_kp1 = RK4_np(mass_string_ode, xk, uk, t, h)
    x_list += x_kp1.flatten().tolist()
    xk = x_kp1
    xk += Ck @ wk

In [17]:
u_list

[[3.289302564456352],
 [2.9143304455734382],
 [2.559423712216641],
 [2.227483124945789],
 [1.9190003163673781],
 [1.6306539913523577],
 [1.367009302277696],
 [1.1328010388330563],
 [0.9208723946122463],
 [0.7312745679554697],
 [0.563151582540111],
 [0.41538396226541563],
 [0.2895973351187242],
 [0.18197083098187972],
 [0.09261105644719769],
 [0.01366404676742039],
 [-0.05205181434193462],
 [-0.09997933746837895],
 [-0.14048587842777605],
 [-0.16826270574733942],
 [-0.18652273576282874],
 [-0.20074084940608494],
 [-0.21144310795526425],
 [-0.21556020609411447],
 [-0.21933974180197768],
 [-0.21154783765784513],
 [-0.20380417156465755],
 [-0.19447432719023183],
 [-0.1861604724545378],
 [-0.1728037158302806],
 [-0.1610938449234819],
 [-0.1482663199093912],
 [-0.13553746942379205],
 [-0.11938088888492306],
 [-0.10513315456413816],
 [-0.09478045138448293],
 [-0.08574017709716915],
 [-0.0759618274370691],
 [-0.06409843045680041],
 [-0.05360195673828133],
 [-0.04387649883910471],
 [-0.03790238

In [18]:
x_list

[[-2, 0],
 [-1.9826689238506545, 0.337883851188942],
 [-1.934617289686878, 0.6081523765847574],
 [-1.863046498782901, 0.8186895390646078],
 [-1.773857601547691, 0.9769983330239574],
 [-1.671298855118988, 1.0899861215832085],
 [-1.5570064996731574, 1.163624258666438],
 [-1.4368774243038982, 1.2038388069140624],
 [-1.316886913675434, 1.2165318635065394],
 [-1.1958298188749448, 1.206362095046335],
 [-1.0761907772924222, 1.177592238551027],
 [-0.9596990742836524, 1.134030010757676],
 [-0.8476855025918736, 1.0790402257138687],
 [-0.7429022190480169, 1.0158649187522126],
 [-0.644753765473668, 0.947074444416752],
 [-0.5546582376670464, 0.8750287577631876],
 [-0.4691245758863931, 0.801077807175922],
 [-0.39056263113519357, 0.726744798109641],
 [-0.32252159827663374, 0.6538898864286646],
 [-0.2597321887850927, 0.5831480817165144],
 [-0.20572877537335316, 0.5156450323527458],
 [-0.15916762471985166, 0.45207941234187016],
 [-0.11711668294635905, 0.392527337313062],
 [-0.07929895096895942, 0.33700

### System formulation
#### plant:

$$
\begin{aligned}
x_{k+1} &=A x_{k}+B u_{k} + C w_{k}\\
y_{k} &=D x_{k} + E w_{k}
\end{aligned}
$$
with $x_k \in \mathbb{R}^{n}, u_k \in \mathbb{R}^{m}, y_k \in \mathbb{R}^{r}, w_k \in \mathbb{R}^{d}$. Initial state $x_0$.


#### model:

$$
\begin{aligned}
\hat{x}_{k+1} &=A \hat{x}_{k}+B u_{k}\\
\hat{y}_{k} &=D \hat{x}_{k}
\end{aligned}
$$

Initial state $0$


### System behabior in N step
Define: $$
\boldsymbol{X}=\left[\begin{array}{llll}
x_{k}^{\top}, x_{k+1}^{\top} & \ldots & x_{k+N}^{\top}
\end{array}\right]^{\top}
$$
$$
\boldsymbol{U}=\left[\begin{array}{llll}
u_{k}^{\top} & u_{k+1}^{\top} & \ldots & u_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$
$$
\boldsymbol{Y}=\left[\begin{array}{llll}
y_{k}^{\top} & y_{k+1}^{\top} & \ldots & y_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$
$$
\boldsymbol{Y}_r=\left[\begin{array}{llll}
yr_{k}^{\top} & yr_{k+1}^{\top} & \ldots & yr_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$

$$
\boldsymbol{W}=\left[\begin{array}{llll}
w_{k}^{\top} & w_{k+1}^{\top} & \ldots & w_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$

$$
\tilde{\boldsymbol{W}}=\left[\begin{array}{llll}
1 & w_{k}^{\top} & \ldots & w_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$

#### Predicted system
$$
\begin{array}{l}
\boldsymbol{X}=\mathcal{A}_{x} x_{k}+\mathcal{B}_{x} \boldsymbol{U} + \mathcal{C}_{x} \boldsymbol{W} \\
\boldsymbol{Y}=\mathcal{A}_{y} x_{k}+\mathcal{B}_{y} \boldsymbol{U} + \mathcal{C}_{y} \boldsymbol{W} + \mathcal{E}_{y} \boldsymbol{W}
\end{array}
$$
with
$$
\mathcal{A}_{x}=\left[\begin{array}{l}
\mathbb{I}_n \\ A^{1}\\  \vdots \\A^{N}
\end{array}\right]
$$

$$
\mathcal{B}_{x}=\left[\begin{array}{ccccc}
0 & 0 & \ldots & \ldots & 0\\
A^{0} B & 0 & \ldots & \ldots & 0 \\
A^{1} B & A^{0} B & \ddots & & \vdots \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & & \ddots & \ddots & 0 \\
A^{N-1} B & A^{N-2} B & \ldots & \ldots & A^{0} B
\end{array}\right]
$$

$$
\mathcal{C}_{x}=\left[\begin{array}{ccccc}
0 & 0 & \ldots & \ldots & 0\\
A^{0} C & 0 & \ldots & \ldots & 0 \\
A^{1} C & A^{0} C & \ddots & & \vdots \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & & \ddots & \ddots & 0 \\
A^{N-1} C & A^{N-2} C & \ldots & \ldots & A^{0} C
\end{array}\right]
$$

$$
\mathcal{A}_{y}=\left[\begin{array}{l}
D \\ DA^{1} \\ \vdots \\DA^{N-1}
\end{array}\right]
$$

$$
\mathcal{B}_{y}=\left[\begin{array}{cccccc}
0 & 0 & \ldots & \ldots & 0 & 0\\
DA^{0} B & 0 & \ldots & \ldots & 0 & 0\\
DA^{1} B & DA^{0} B & \ddots & & \vdots & 0\\
\vdots & \ddots & \ddots & \ddots & \vdots & 0\\
\vdots & & \ddots & \ddots & 0 & 0\\
DA^{N-2} B & DA^{N-3} B & \ldots & \ldots & DA^{0} B & 0
\end{array}\right]
$$

$$
\mathcal{C}_{y}=\left[\begin{array}{cccccc}
0 & 0 & \ldots & \ldots & 0 & 0\\
DA^{0} C & 0 & \ldots & \ldots & 0 & 0\\
DA^{1} C & DA^{0} C & \ddots & & \vdots & 0\\
\vdots & \ddots & \ddots & \ddots & \vdots & 0\\
\vdots & & \ddots & \ddots & 0 & 0\\
DA^{N-2} C & DA^{N-3} C & \ldots & \ldots & DA^{0} C & 0
\end{array}\right]
$$

$$
\mathcal{E}_{y} = \left[
  \begin{array}{ccc}
    E & & & & \\
    & \ddots & & & \\
    & & \ddots & & \\
    & & & \ddots & \\
    & & & & E
  \end{array} 
  \right]
$$


#### System with extended $\tilde{\boldsymbol{W}}$

$$
\begin{array}{l}
\boldsymbol{X}=\mathcal{B}_{x} \boldsymbol{U} + \tilde{\mathcal{C}}_{x} \tilde{\boldsymbol{W}} \\
\end{array}
$$

$$
\tilde{\mathcal{C}}_{x}=\left[\begin{array}{cccccc}
x_k & 0 & 0 & \ldots & \ldots & 0\\
A x_k & A^{0} C & 0 & \ldots & \ldots & 0 \\
A^2 x_k& A^{1} C & A^{0} C & \ddots & & \vdots \\
\vdots & \vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & \vdots & & \ddots & \ddots & 0 \\
A^N x_k & A^{N-1} C & A^{N-2} C & \ldots & \ldots & A^{0} C
\end{array}\right]
$$

#### puried-output-based (POB) affine control laws

$$
y - \hat{y} = \left(
\left[\begin{array}{c|cccc}
DA^0 x_0 & 0 & \dots & 0 & 0\\
DA^1 x_0 & DC & \ddots & 0 & 0\\
\vdots & \vdots & \ddots & \ddots & \vdots\\
DA^{N-1} x_0 & DA^{N-2}C & \dots & DA^{0}C & 0\\
\end{array}\right]
+
\left[\begin{array}{c|cccc}
0 & E & 0 & \dots & 0\\
0 & 0 & E & \ddots & 0\\
\vdots &  & \ddots & \ddots & 0\\
0 & 0 & \dots & 0 & E\\
\end{array}\right]
\right)
\left[\begin{array}{c}
1\\
w_0\\
w_1\\
\vdots\\
w_{N-1}\\
\end{array}\right]
$$




$u_{k}=h_{k}+\sum_{\tau=0}^{k} H_{k \tau} v_{\tau}$, where $v_{k} = y_k - \hat{y}_k$

$$
\begin{array}{rll}
u_k &= h_0 + H_{00} (y_k - \hat{y}_k) &= h_0 + H_{00} (Dx_0 +Ew_0)\\
u_{k+1} &= h_1 + H_{10} (y_k - \hat{y}_k) + H_{11} (y_{k+1} - \hat{y}_{k+1}) &= h_1 + H_{10} (Dx_0 +Ew_0) + H_{11} (DAx_0 + DCw_0 + E w_1) \\
\vdots &  & \vdots \\
u_{k+N-1} &= h_{N-1}+\sum_{\tau=0}^{N-1} H_{(N-1) \tau} v_{\tau} &= h_{N-1} +H_{(N-1) 0}(Dx_0 +Ew_0) + \dots + H_{(N-1) (N-1)} (DA^{N-1} x_0 + DA^{N-2}Cw_0 + \dots + DA^{0}Cw_{N-2}) + Ew_{N-1}
\end{array}
$$

$\rightarrow$

$$\boldsymbol{U} = 
\left[\begin{array}{c|cccc}
h_0 & H_{00} & \dots & 0 & 0\\
h_1 & H_{10} & \ddots & 0 & 0\\
\vdots & \vdots & \ddots & \ddots & \vdots\\
h_{N-1} & H_{(N-1)0} & \dots & H_{(N-1)(N-2)} & H_{(N-1)(N-1)}\\
\end{array}\right]
\left(
\left[\begin{array}{c|cccc}
0 & 0 & \dots & 0 & 0\\
DA^0 x_0 & 0 & \dots & 0 & 0\\
DA^1 x_0 & DC & \ddots & 0 & 0\\
\vdots & \vdots & \ddots & \ddots & \vdots\\
DA^{N-1} x_0 & DA^{N-2}C & \dots & DA^{0}C & 0\\
\end{array}\right]
+
\left[\begin{array}{c|cccc}
1 & 0 & \dots & 0 & 0\\
0 & E & 0 & \dots & 0\\
0 & 0 & E & \ddots & 0\\
\vdots &  & \ddots & \ddots & 0\\
0 & 0 & \dots & 0 & E\\
\end{array}\right]
\right)
\left[\begin{array}{c}
1\\
w_0\\
w_1\\
\vdots\\
w_{N-1}\\
\end{array}\right]
$$

$\rightarrow$
$$\boldsymbol{U} = \mathcal{H}(\tilde{\mathcal{C}}_y + \tilde{\mathcal{E}}_y) \tilde{\boldsymbol{W}} = H\tilde{\boldsymbol{W}} $$, where $ \tilde{\mathcal{C}}_y  = \left[\begin{array}{ccccc}
0 & & & & \\
D & 0 & & & \\
& D & 0 & & \\
& & \ddots & \ddots & \\
& & & D & 0
\end{array}\right] \tilde{\mathcal{C}}_x  = \tilde{\mathcal{D}} \tilde{C}_x$


#### Matrix shape
$\boldsymbol{X} \in \mathbb{R}^{(N+1)n \times 1} $,
$\boldsymbol{U} \in \mathbb{R}^{Nm \times 1} $,
$\boldsymbol{Y} \in \mathbb{R}^{Nr \times 1} $,
$\boldsymbol{W} \in \mathbb{R}^{Nd \times 1} $
$\tilde{\boldsymbol{W}} \in \mathbb{R}^{(Nd+1) \times 1} $

$\mathcal{A}_{x} \in \mathbb{R}^{(N+1)n \times n}$,
$\mathcal{B}_{x} \in \mathbb{R}^{(N+1)n \times Nm}$,
$\mathcal{C}_{x} \in \mathbb{R}^{(N+1)n \times Nd}$,
$\mathcal{A}_{y} \in \mathbb{R}^{Nr \times n}$,
$\mathcal{B}_{y} \in \mathbb{R}^{Nr \times Nm}$,
$\mathcal{C}_{y} \in \mathbb{R}^{Nr \times Nd}$,
$\mathcal{E}_{y} \in \mathbb{R}^{Nr \times Nd}$

$\tilde{\mathcal{C}}_{x} \in \mathbb{R}^{(N+1)n \times (Nd+1)}$, $\tilde{\mathcal{C}}_{y} \in \mathbb{R}^{(Nr + 1) \times (Nd+1)}$, $\tilde{\mathcal{E}}_{y} \in \mathbb{R}^{(Nr + 1) \times (Nd+1)}$, $\tilde{\mathcal{D}} \in \mathbb{R}^{(Nr + 1) \times (N+1)n}$

$\mathcal{H} \in \mathbb{R}^{Nm \times (Nr + 1)}$


### Cost
#### Weighted stage cost
$\beta^{t} (x_{k+t}^{\top} Q x_{k+t} + u_{k+t}^{\top} R u_{k+t} )$
#### Cost
$J_{N}\left(x, \pi_{N}\right):=\sup _{\mathbb{P} \in \mathcal{P}_{\infty}} \mathbb{E}_{\mathbb{P}}\left\{\sum_{t=0}^{N-1} \beta^{t}\left[\boldsymbol{x}_{t}^{\top} Q \boldsymbol{x}_{t}+\boldsymbol{u}_{t}^{\top} R \boldsymbol{u}_{t}\right]+\beta^{N} \boldsymbol{x}_{N}^{\top} Q_{f} \boldsymbol{x}_{N}\right\}$
#### Reformulation with known first and second moments
$$\boldsymbol{U} = \mathcal{H}(\tilde{\mathcal{C}}_y + \tilde{\mathcal{E}}_y) \tilde{\boldsymbol{W}} = H\tilde{\boldsymbol{W}}$$ and $$\boldsymbol{X}=\mathcal{B}_{x} \boldsymbol{U} + \tilde{\mathcal{C}}_{x} \tilde{\boldsymbol{W}}$$

$\tilde{J}_{N}(x, H):=\operatorname{Tr}\left\{\left[(\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H)^{\top} J_{x} (\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H) + H^{\top} J_u H \right]M_w\right\} + \mu_w^{\top} \left[ (\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H)^{\top} J_{x} (\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H) + H^{\top} J_u H \right]\mu_w$ with $J_x = \left[ \begin{array}{ccccc} \beta^{0}Q&&&&\\ &\beta^{1}Q&&&\\ &&\ddots&& \\ &&&\beta^{N-1}Q&\\ &&&&\beta^{N}Q_f \end{array}\right]$, $J_u = \left[ \begin{array}{cccc} \beta^{0}R&&&\\ &\beta^{1}R&&\\ &&\ddots& \\ &&&\beta^{N-1}R\end{array}\right]$, $M_{w}:=\left(1, \mu^{\top}, \ldots, \mu^{\top}\right)^{\top}\left(1, \mu^{\top}, \ldots, \mu^{\top}\right)+\operatorname{diag}\left(0, \mathbb{I}_{N} \otimes \mathbb{I}_{d}\right)$ and $\mu_w = \left[ \begin{array}{c} 1\\ \mu \\ \vdots \\ \mu \end{array} \right]$

### Chance constraint using wasserstein metric
Constraint on ith element of state: $[x_{k+n}]_i < v_{ubi}$


$\sup _{\mathbb{Q} \in \mathbb{B}_{\varepsilon}\left(\widehat{\mathbb{P}}_{N}\right)} \mathbb{E}^{\mathrm{Q}}[\ell(\xi)]$ with $l(w) = \max_{k \le N-1} \langle a_k, w\rangle + b_k$

equivalent to
$\begin{array}{lll}\inf _{\lambda, s_{i}, \gamma_{i k}} & \lambda \varepsilon+\frac{1}{N} \sum_{i=1}^{N} s_{i} \\ \text { s.t. } & b_{k}+\left\langle a_{k}, \widehat{\xi}_{i}\right\rangle+\left\langle\gamma_{i k}, d-C \widehat{\xi}_{i}\right\rangle \leq s_{i} & \forall i \leq N, \forall k \leq K \\ & \left\|C^{\top} \gamma_{i k}-a_{k}\right\|_{*} \leq \lambda & \forall i \leq N, \forall k \leq K \\ & \gamma_{i k} \geq 0 & \forall i \leq N, \forall k \leq K\end{array}$


#### Define $a_k$ and $b_k$
For example: Constraint on $[x_{k+1}]_1$

We have $x_{k+1} = Ax_0 + Bu_0 + Cw_0$ and $u_0 = h_0 + H_{00}(Dx_0 + Ew_0)$ $\rightarrow$ $[x_{k+1}]_1 = e_1^{\top} (Ax_0 + B(h_0 + H_{00}(Dx_0 + Ew_0)) + Cw_0) = \langle e_1^{\top} (BH_{00}E + C), w_0 \rangle  + e_1^{\top} (Ax_0 + B(h_0 + H_{00}Dx_0))$ 

Similiarly, $x_{k+N} = A^N x_0 + A^{N-1}Bu_0 + \dots + A^{0}Bu_{N-1} + A^{N-1}Cw_0 + \dots + A^{0}Cw_{N-1}$ and $u_{n}=h_{n}+\sum_{\tau=0}^{n} H_{n \tau} v_{\tau}$, where $v_{\tau} = y_{\tau} - \hat{y}_{\tau} = DA^{\tau}x_0 + DA^{\tau-1}Cw_0 + \dots + DA^{0}C w_{\tau-1} + E w_{\tau}$

$\rightarrow$ $[x_{k+N}]_i = e^{\top}_i (A^Nx_0 + \sum_{n=0}^{N-1}A^{n}B \underbrace{(h_{N-1-n} + \sum_{\tau = 0}^{N - 1 - n} ( H_{n\tau} \underbrace{( DA^{\tau}x_0 + \sum_{\kappa = 0}^{\tau-1} DA^{\kappa}Cw_{\tau-1 - \kappa} + Ew_{\tau})}_{v_\tau}))}_{u_{N- 1 - n}} + \sum_{n=0}^{N-1} A^{n}Cw_{N-1-n}) $

$ [x_{k+N}]_i = \langle e_i^{\top}(BH_{(N-1)(N-1)}E + C), w_{N-1} \rangle + e^{\top}_i (A^Nx_0 + \sum_{n=0}^{N-1}A^{n}B(h_{N-1-n} + \sum_{\tau = 0}^{N - 1 - n} ( H_{n\tau}( DA^{\tau}x_0 + \sum_{\kappa = 0}^{\tau-1} DA^{\kappa}Cw_{\tau-1 - \kappa}))) + \sum_{n=1}^{N-1} A^{n}Cw_{N-1-n})$

In [19]:
# def stack_system(A, B, C, D, E, x_init, N):
#     '''
#     Stack system matrix for N prediction horizon
    
#     x_next = A x_k + B u_k + C w_k
#     y_k = D x_k + E w_k
    
#     '''

#     n = np.shape(A)[1]    #  Dimension of state
#     m = np.shape(B)[1]    #  Dimension of input
#     d = np.shape(C)[1]    #  Dimension of disturbance
#     r = np.shape(D)[0]    #  Dimension of output
    
# #     print(Nx,Nu,Nw,Ny)
#     Nx = (N+1) * n
#     Nu = N * m
#     Ny = N * r
#     Nw = N * d
    
#     Ax = np.zeros([Nx,n])
#     Bx = np.zeros([Nx,Nu])
#     Cx = np.zeros([Nx,Nw])
#     Ay = np.zeros([Ny,n])
#     By = np.zeros([Ny,Nu])
#     Cy = np.zeros([Ny,Nw])
#     Ey = np.zeros([Ny,Nw])
#     Cx_tilde = np.zeros([Nx, Nw + 1])
#     Cy_tilde = np.zeros([Ny + 1, Nw + 1])
#     Ey_tilde = np.zeros([Ny + 1, Nw + 1])
#     D_tilde = np.zeros([Ny + 1, Nx])
# #     H

# # Ax
#     for i in range(N+1):
#         Ax[i*n:(i+1)*n,:] = matrix_power(A,i)
# # Bx
#     for i in range(N):
#         mat_temp = B
#         for j in range (i+1):
#             Bx[(i + 1) * n : (i + 2) * n, (i-j) * m: (i-j+1) * m ] = mat_temp
#             mat_temp = A @ mat_temp
# # Cx
#     for i in range(N):
#         mat_temp = C
#         for j in range (i+1):
#             Cx[(i + 1) * n : (i + 2) * n, (i-j) * d: (i-j+1) * d ] = mat_temp
#             mat_temp = A @ mat_temp
# # Ay
#     for i in range(N):
#         Ay[i*r:(i+1)*r,:] = D @ matrix_power(A,i) 
# # By
#     for i in range(N):
#         mat_temp = B
#         for j in range (i+1):
#             By[(i + 1) * r : (i + 2) * r, (i-j) * m: (i-j+1) * m ] = D @ mat_temp
#             mat_temp = A @ mat_temp
# # Cy
#     for i in range(N):
#         mat_temp = C
#         for j in range (i+1):
#             Cy[(i + 1) * r : (i + 2) * r, (i-j) * d: (i-j+1) * d ] = D @ mat_temp
#             mat_temp = A @ mat_temp
# # Ey
#     for i in range(N):
#         Ey[i * r : (i+1)*r, i * d : (i+1) * d] = E
# # Cx_tilde
#     Cx_tilde[ : , 0 : 1 ] = Ax @ x_init
#     Cx_tilde[ : , 1 : ] = Cx
# # Cy_tilde
#     Cy_tilde[ r : , 0 : 1 ] = Ay @ x_init
#     Cy_tilde[ r : , 1 : ] = Cy
# # Ey_tilde
#     Ey_tilde[0, 0] = 1
#     Ey_tilde[1:, 1:] = Ey
# # D_tilde
#     for i in range(N):
#         D_tilde[1 + i * r : 1 + (i+1) * r, i * d : (i+1) * d] = D
        
#     return n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde 

In [20]:
# Ck = np.array([[1e-3, 0],[0, 0]])
# D = np.array([[1, 0]])
# E = np.array([[0,1e-3]])

# N = 5
# n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde  = stack_system(Ak,Bk,Ck,D,E, x_init, N)

In [21]:
Ax

array([[ 1.        ,  0.        ],
       [ 0.        ,  1.        ],
       [ 0.99137931,  0.0862069 ],
       [-0.0862069 ,  0.86206897],
       [ 0.97540131,  0.15978002],
       [-0.15978002,  0.73573127],
       [ 0.95321854,  0.22182772],
       [-0.22182772,  0.62047696],
       [ 0.92587806,  0.2734048 ],
       [-0.2734048 ,  0.51577085],
       [ 0.89432697,  0.31551087],
       [-0.31551087,  0.42106066]])

In [22]:
Bx

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00431034, 0.        , 0.        , 0.        , 0.        ],
       [0.04310345, 0.        , 0.        , 0.        , 0.        ],
       [0.007989  , 0.00431034, 0.        , 0.        , 0.        ],
       [0.03678656, 0.04310345, 0.        , 0.        , 0.        ],
       [0.01109139, 0.007989  , 0.00431034, 0.        , 0.        ],
       [0.03102385, 0.03678656, 0.04310345, 0.        , 0.        ],
       [0.01367024, 0.01109139, 0.007989  , 0.00431034, 0.        ],
       [0.02578854, 0.03102385, 0.03678656, 0.04310345, 0.        ],
       [0.01577554, 0.01367024, 0.01109139, 0.007989  , 0.00431034],
       [0.02105303, 0.02578854, 0.03102385, 0.03678656, 0.04310345]])

In [23]:
Cx

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.00000000e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 9.91379310e-04,  0.00000000e+00,  1.00000000e-03,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.

In [24]:
Ay

array([[1.        , 0.        ],
       [0.99137931, 0.0862069 ],
       [0.97540131, 0.15978002],
       [0.95321854, 0.22182772],
       [0.92587806, 0.2734048 ]])

In [25]:
By

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00431034, 0.        , 0.        , 0.        , 0.        ],
       [0.007989  , 0.00431034, 0.        , 0.        , 0.        ],
       [0.01109139, 0.007989  , 0.00431034, 0.        , 0.        ],
       [0.01367024, 0.01109139, 0.007989  , 0.00431034, 0.        ]])

In [26]:
Cy

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.001     , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00099138, 0.        , 0.001     , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.0009754 , 0.        , 0.00099138, 0.        , 0.001     ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00095322, 0.        , 0.0009754 , 0.        , 0.00099138,
        0.        , 0.001     , 0.        , 0.        , 0.        ]])

In [27]:
Ey

array([[0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   ],
       [0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   ,
        0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   ,
        0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.001]])

In [28]:
Cx_tilde

array([[-2.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.98275862e+00,  1.00000000e-03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.72413793e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.95080262e+00,  9.91379310e-04,  0.00000000e+00,
         1.00000000e-03,  0.00000000e+00,  0.00000000e+00,
  

In [29]:
Cy_tilde

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-2.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.98275862e+00,  1.00000000e-03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.95080262e+00,  9.91379310e-04,  0.00000000e+00,
         1.00000000e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.90643707e+00,  9.75401308e-04,  0.00000000e+00,
         9.91379310e-04,  0.00000000e+00,  1.00000000e-03,
  

In [30]:
Ey_tilde

array([[1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.001]])

In [31]:
D_tilde

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [32]:
# Decision varibale H
# h vector
H_cal_dec = cp.Variable((Nu , 1))
print(H_cal_dec)
# \mathcal{H} matrix
for i in range(N):
    H_col = cp.Variable((Nu - (i * m) , r))
    if i > 0:
        H_col = cp.vstack([np.zeros([i * m, r]), H_col])
    print(H_col)
    H_cal_dec = cp.hstack([H_cal_dec,H_col])
print(H_cal_dec)

var0
var1
Vstack([[0.]], var3)
Vstack([[0.]
 [0.]], var6)
Vstack([[0.]
 [0.]
 [0.]], var9)
Vstack([[0.]
 [0.]
 [0.]
 [0.]], var12)
Hstack(Hstack(Hstack(Hstack(Hstack(var0, var1), Vstack([[0.]], var3)), Vstack([[0.]
 [0.]], var6)), Vstack([[0.]
 [0.]
 [0.]], var9)), Vstack([[0.]
 [0.]
 [0.]
 [0.]], var12))


In [33]:
print(np.shape(H_cal_dec))

(5, 6)


In [34]:
H_cal_dec

Expression(AFFINE, UNKNOWN, (5, 6))

In [35]:
# H_cal_dec = cp.Variable((Nu , 1 + Ny)) # Ny = N * r
# H = H_cal_dec @ (Cy_tilde + Ey_tilde)
H = cp.Variable((Nu , Nw + 1))

In [36]:
# Define loss function
beta = 0.95


Jx = np.zeros([(N+1)* n, (N+1)* n])
for i in range(N):
    Jx[i * n : (i+1) * n, i * n : (i+1) * n] = beta**i * Q
Jx[ N * n: , N * n:] = beta ** N * Qf

Ju = np.zeros([N * m, N * m])
for i in range(N):
    Ju[i * m : (i+1) * m, i * m : (i+1) * m] = beta**i * R

    
mu =np.zeros([d,1])
mu_w = np.vstack([1]+ [mu] * N)
M_w = mu_w @ mu_w.T + np.diag([0] + [1] * N * d)

# loss_func = cp.trace( ((Cx_tilde+Bx @ H).T @ Jx @ (Cx_tilde+Bx @ H) +  H.T @ Ju @ H ) @ M_w ) + mu_w.T @ ((Cx_tilde+Bx @ H).T @ Jx @ (Cx_tilde+Bx @ H) +  H.T @ Ju @ H ) @ mu_w
# loss_func = cp.trace( ((Cx_tilde+Bx @ H).T @ Jx @ (Cx_tilde+Bx @ H) +  H.T @ Ju @ H ) @ M_w ) 
loss_func = cp.trace( (Cx_tilde.T @ Jx @ Cx_tilde +  H.T @ (Ju + Bx.T @ Jx @ Bx)@ H + 2 * Cx_tilde.T @ Jx @ Bx @ H) @ M_w ) 
# loss_func = cp.trace( (H.T @ (Ju + Bx.T @ Jx @ Bx)@ H ) @ M_w ) 
# loss_func = cp.trace( cp.quad_form(H, Ju + Bx.T @ Jx @ Bx) @ M_w ) 
# loss_func = cp.trace( cp.quad_form(H,Ju + Bx.T @ Jx @ Bx)@ M_w ) 

In [37]:
H_new_matrix = []
for i in range(Nw+1):
    H_new_matrix += [cp.Variable([Nu,1])]
H_new = cp.hstack(H_new_matrix)
print(H_new.shape)

(5, 11)


In [38]:
np.linalg.eig((Ju + Bx.T @ Jx @ Bx))

(array([1.01272643, 0.95732504, 0.90739038, 0.86047056, 0.81605703]),
 array([[-0.9849619 , -0.16455627,  0.04874987, -0.01887036, -0.0062176 ],
        [-0.15790468,  0.98100613,  0.10807906, -0.03049136, -0.00907063],
        [-0.0637849 ,  0.09663711, -0.99082099, -0.06812168, -0.01503216],
        [-0.02753592,  0.03316662, -0.06298989,  0.99650455, -0.03395144],
        [-0.0094572 ,  0.01046177, -0.01576115,  0.03243917,  0.99924992]]))

In [39]:
eigval,eigvec = np.linalg.eig((Ju + Bx.T @ Jx @ Bx))

In [40]:
eigval_mat =  np.diag(eigval)

In [41]:
Ju + Bx.T @ Jx @ Bx - eigvec @ eigval_mat @ np.linalg.inv(eigvec)

array([[ 4.44089210e-16,  2.37657116e-16,  9.28077060e-17,
         1.30104261e-17,  1.73472348e-18],
       [ 1.92554306e-16, -3.33066907e-16,  1.50053581e-16,
         1.13624388e-16,  1.29453739e-16],
       [ 8.23993651e-17,  1.25767452e-16, -6.66133815e-16,
        -8.28330460e-17, -2.26381414e-16],
       [ 8.23993651e-18,  1.21430643e-16, -2.29417180e-16,
         4.44089210e-16,  2.67147415e-16],
       [-2.38524478e-18,  1.11239143e-16, -1.93855348e-16,
         3.25043811e-16,  2.22044605e-16]])

In [42]:
Ju + Bx.T @ Jx @ Bx - eigvec @ eigval_mat @ eigvec.T

array([[ 6.66133815e-16,  0.00000000e+00,  1.16226473e-16,
         6.37510877e-17, -3.85975973e-17],
       [-5.55111512e-17, -3.33066907e-16, -2.42861287e-17,
         3.97251676e-16,  4.77048956e-18],
       [ 1.02348685e-16, -3.90312782e-17, -2.22044605e-16,
        -2.50190493e-15,  5.42101086e-18],
       [ 6.37510877e-17,  3.90312782e-16, -2.50233861e-15,
         1.11022302e-16,  1.25984292e-15],
       [-3.85975973e-17,  4.98732999e-18,  7.15573434e-18,
         1.25984292e-15,  2.22044605e-16]])

In [43]:
loss_func = 0

N_eig = np.shape(eigval)[0]
I = np.diag([1]*N_eig)
for i in range(N_eig):
    # Tr(H.T @ (Ju + Bx.T @ Jx @ Bx)@ H ) @ M_w
    loss_func += eigval[i] * cp.quad_form(H_new_matrix[i], I) # When M_w is identity matrix. Otherwise reformulate system matrix or this line

loss_func += cp.trace(2 * Cx_tilde.T @ Jx @ Bx @ H)
loss_func += eigval[0] * cp.quad_form(H_new_matrix[0], I) +  2 * mu_w.T @  Cx_tilde.T @ Jx @ Bx @ H @ mu_w


In [44]:
np.ones(3)

array([1., 1., 1.])

In [45]:
N_sample = 6

w_sample = []
for i in range(N_sample):
    w_temp = np.random.randn(N*d)
    w_sample += [w_temp]
W_sample_matrix = np.array(w_sample).T

W_sample_matrix_ext = np.vstack( [np.ones([1, N_sample]),W_sample_matrix])

In [46]:
w_sample[0]

array([ 0.2843304 , -0.74072901, -0.5728423 , -0.65147159,  1.76064824,
        0.38803277, -0.97364452, -1.51796105, -3.45870599,  1.00319101])

In [47]:
W_sample_matrix

array([[ 0.2843304 ,  1.70039576,  0.05284195,  0.22970177,  0.99153464,
        -0.13896296],
       [-0.74072901,  0.50755833, -0.97312626, -0.29945658,  1.24180091,
         0.83844907],
       [-0.5728423 , -0.77803194, -0.26004654,  0.22200064,  0.46526977,
         1.35222026],
       [-0.65147159, -1.76553057,  1.09689212, -1.58127818, -0.03573322,
        -0.29268077],
       [ 1.76064824, -1.68611675, -0.05634167, -0.0872003 , -0.16124329,
         2.01512972],
       [ 0.38803277, -1.4879493 , -0.3849617 , -0.16814053, -1.14008304,
        -1.29666375],
       [-0.97364452, -0.14412195,  1.814566  ,  2.26046551, -0.28540553,
         0.32192103],
       [-1.51796105,  0.06809123, -0.85045734, -0.72191519,  0.28645025,
         0.68393682],
       [-3.45870599, -1.59873644, -1.42522313, -0.67434758, -1.68819389,
        -0.02101449],
       [ 1.00319101, -0.03884549, -0.12434205, -0.21699575, -1.76404619,
        -0.01217894]])

In [48]:
W_sample_matrix_ext

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ],
       [ 0.2843304 ,  1.70039576,  0.05284195,  0.22970177,  0.99153464,
        -0.13896296],
       [-0.74072901,  0.50755833, -0.97312626, -0.29945658,  1.24180091,
         0.83844907],
       [-0.5728423 , -0.77803194, -0.26004654,  0.22200064,  0.46526977,
         1.35222026],
       [-0.65147159, -1.76553057,  1.09689212, -1.58127818, -0.03573322,
        -0.29268077],
       [ 1.76064824, -1.68611675, -0.05634167, -0.0872003 , -0.16124329,
         2.01512972],
       [ 0.38803277, -1.4879493 , -0.3849617 , -0.16814053, -1.14008304,
        -1.29666375],
       [-0.97364452, -0.14412195,  1.814566  ,  2.26046551, -0.28540553,
         0.32192103],
       [-1.51796105,  0.06809123, -0.85045734, -0.72191519,  0.28645025,
         0.68393682],
       [-3.45870599, -1.59873644, -1.42522313, -0.67434758, -1.68819389,
        -0.02101449],
       [ 1.00319101, -0.03884549, -0.12434205, -0.

In [49]:
Cx_tilde

array([[-2.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.98275862e+00,  1.00000000e-03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.72413793e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.95080262e+00,  9.91379310e-04,  0.00000000e+00,
         1.00000000e-03,  0.00000000e+00,  0.00000000e+00,
  

In [50]:
# H = H_dec @ (Cy_tilde + Ey_tilde)

In [51]:
X_constraint = (Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) + Cx_tilde) @ W_sample_matrix_ext

In [52]:
# select i th element from X_

In [53]:
print(np.shape(X_constraint))

(12, 6)


In [54]:
constraint = []
# constraint += [H_new == np.linalg.inv(eigvec) @ H ]
constraint += [H_new == eigvec.T @ H ]
constraint += [H == H_cal_dec @ (Cy_tilde + Ey_tilde) ]

In [55]:
# d_supp = np.ones([N*d, 1])
# C_supp = np.diag([1]*N*d)

# epsilon = 0.2

# lambda_var = cp.Variable()

# gamma_matrix = []
# for i in range(N_sample):
#     for j in range(N):
#         gamma_var = cp.Variable([N*d,1])
#         gamma_matrix += [gamma_var]

# si_var = cp.Variable([N_sample,1])
# for i in range(N_sample):
#     for j in range(N):
#         constraint_temp = X_constraint[n * (j+1),i] + gamma_matrix[i * N_sample + j].T @ (d_supp - C_supp @ W_sample_matrix[:, i])
#         constraint += [constraint_temp <= si_var[i,0]]
        
        
# ak_matrix = (Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) + Cx_tilde)[:,1:]
# for i in range(N_sample):
#     for j in range(N):
#         constraint_temp = C_supp.T @ gamma_matrix[i * N_sample + j] - ak_matrix[n * (j+1):n * (j+1)+1,:].T
#         constraint += [cp.norm_inf(constraint_temp) <= lambda_var]
        
# for i in range(N_sample):
#     for j in range(N):
#         constraint += [gamma_matrix[i * N_sample + j] >= 0]

        
# constraint += [lambda_var * epsilon + 1/N_sample * cp.sum(si_var) <= 0.5]
# # supp_constraint_matrix = 

In [56]:
# len(gamma_matrix)

In [57]:
# i * N_sample + j

In [58]:
# gamma_matrix[i * N_sample + j].T 

In [59]:
i_th_state = 1 # 0 for first element, 1 for second element
i_state_ub = 0.01

d_supp = 3 * np.ones([N*d, 1])
C_supp = np.diag([1]*N*d)

epsilon = 5 # select 20 best

lambda_var = cp.Variable()

gamma_matrix = []
for i in range(N_sample):
    for j in range(N):
        gamma_var = cp.Variable([N*d,1])
        gamma_matrix += [gamma_var]

si_var = cp.Variable([N_sample,1])
for i in range(N_sample):
    for j in range(N):
        constraint_temp = X_constraint[n * (j+1) + i_th_state,i] + gamma_matrix[i * N + j].T @ (d_supp - C_supp @ W_sample_matrix[:, i])
        constraint += [constraint_temp <= si_var[i,0]]
        
        
ak_matrix = (Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) + Cx_tilde)[:,1:]
for i in range(N_sample):
    for j in range(N):
        constraint_temp = C_supp.T @ gamma_matrix[i * N + j] - ak_matrix[n * (j+1) + i_th_state:n * (j+1)+i_th_state + 1,:].T
        constraint += [cp.norm_inf(constraint_temp) <= lambda_var]
        
for i in range(N_sample):
    for j in range(N):
        constraint += [gamma_matrix[i * N + j] >= 0]

        
constraint += [lambda_var * epsilon + 1/N_sample * cp.sum(si_var) <= i_state_ub]
# supp_constraint_matrix = 

In [60]:
obj = cp.Minimize(loss_func)
prob = cp.Problem(obj,constraint)
prob.solve()

189.1156444456611

In [61]:
lambda_var.value

array(4.93601416e-05)

In [62]:
si_var.value

array([[0.00986468],
       [0.00974107],
       [0.00989686],
       [0.00970572],
       [0.00965288],
       [0.00967005]])

In [63]:
np.sum(si_var.value)/N_sample + lambda_var.value * epsilon

0.010002010139647678

In [64]:
H_new_matrix[0].value

array([[ 4.7437227 ],
       [-3.82576882],
       [ 3.62580562],
       [-3.61460635],
       [-3.70232401]])

In [65]:
H_cal_dec.value

array([[-5.93561219, -1.08038319,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-1.9119281 , -0.08047383,  1.11738659,  0.        ,  0.        ,
         0.        ],
       [-0.58898475,  0.1363148 ,  0.39207966,  1.19125979,  0.        ,
         0.        ],
       [-1.73244794, -0.59149607,  0.1890501 ,  0.33072153,  1.25506537,
         0.        ],
       [-0.20867743,  0.16470423,  0.12464392,  0.18822815,  0.19265105,
         1.31720235]])

In [66]:
np.trace(Cx_tilde.T @ Jx @ Cx_tilde) + mu_w.T @ Cx_tilde.T @ Jx @ Cx_tilde @mu_w

array([[417.51649692]])

In [67]:
np.trace(((Cx_tilde+Bx @ H.value).T @ Jx @ (Cx_tilde+Bx @ H.value) + H.value.T @ Ju @ H.value)@ M_w) + mu_w.T @ ((Cx_tilde+Bx @ H.value).T @ Jx @ (Cx_tilde+Bx @ H.value) + H.value.T @ Ju @ H.value) @ mu_w

array([[592.87065391]])

In [68]:
H.value @ W_sample_matrix_ext

array([[-3.77404553, -3.77539416, -3.77379445, -3.77452227, -3.77618742,
        -3.77575165],
       [-3.96683896, -3.96660195, -3.96512532, -3.96797446, -3.96552026,
        -3.96703811],
       [-3.96305567, -3.96357396, -3.96331422, -3.9631617 , -3.96201525,
        -3.96307926],
       [-3.96183137, -3.96359372, -3.96297211, -3.96261544, -3.96067413,
        -3.95787695],
       [-3.95703494, -3.96031726, -3.95729321, -3.95591896, -3.95922169,
        -3.95341205]])

In [69]:
Ax @ x_init +  Bx @ H.value @ W_sample_matrix_ext[:,0:1]  + Cx @ W_sample_matrix[:,0:1]

array([[-2.        ],
       [ 0.        ],
       [-1.99874173],
       [ 0.00973942],
       [-1.99834288],
       [ 0.00971693],
       [-1.9955996 ],
       [ 0.00982624],
       [-1.99559957],
       [ 0.00973675],
       [-1.99807165],
       [ 0.00986635]])

In [70]:
Ck

array([[0.001, 0.   ],
       [0.   , 0.   ]])

In [71]:
H_cal_dec.value[0,0]

-5.935612185683641

In [72]:
H_cal_dec.value[0,1]

-1.0803831921394405

In [73]:
N_sim = 100


t0 = 0

xk = x_init
uk = 0
t = t0
h = delta_t



x_list = []
x_list += [xk.flatten().tolist()]
u_list = []
for i in range(N_sim):
    wk = np.random.randn(d,1)
    uk = H_cal_dec.value[0,0] + H_cal_dec.value[0,1] * (D @ xk + E @ wk)
    u_list +=  uk.flatten().tolist() 
    print(xk,uk)
    x_kp1 = RK4_np(mass_string_ode, xk, uk, t, h)
    x_list += x_kp1.flatten().tolist()
    xk = x_kp1
    xk += Ck @ wk





[[-2]
 [ 0]] [[-3.77292636]]
[[-2.0001691 ]
 [ 0.01052563]] [[-3.77287302]]
[[-1.99923509]
 [ 0.01955567]] [[-3.77721304]]
[[-1.99725465]
 [ 0.02699928]] [[-3.77981866]]
[[-1.99402946]
 [ 0.033068  ]] [[-3.7810564]]
[[-1.98775094]
 [ 0.03790759]] [[-3.78654828]]
[[-1.98325843]
 [ 0.04121453]] [[-3.79252512]]
[[-1.97856981]
 [ 0.04335235]] [[-3.79758181]]
[[-1.97556546]
 [ 0.04451366]] [[-3.79944181]]
[[-1.97108413]
 [ 0.04514322]] [[-3.80632545]]
[[-1.96671943]
 [ 0.04494771]] [[-3.81084413]]
[[-1.96154972]
 [ 0.04416622]] [[-3.81837807]]
[[-1.95606906]
 [ 0.04266862]] [[-3.82175064]]
[[-1.95161377]
 [ 0.04072199]] [[-3.82762724]]
[[-1.9460891 ]
 [ 0.03836987]] [[-3.83529218]]
[[-1.94288432]
 [ 0.03548855]] [[-3.83536897]]
[[-1.94057729]
 [ 0.03272095]] [[-3.83654731]]
[[-1.93793834]
 [ 0.03008287]] [[-3.84050775]]
[[-1.93501378]
 [ 0.02739596]] [[-3.84456598]]
[[-1.93309459]
 [ 0.02463623]] [[-3.84597837]]
[[-1.93198985]
 [ 0.02203   ]] [[-3.84656843]]
[[-1.92959479]
 [ 0.01966882]] [